In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn

import sys
from crm_model import crm_model
from utils import batch_gen
from evaluation_metrics import get_trec_eval_metrics
from tensorflow.keras.backend import set_session

import random
from tensorflow import set_random_seed

In [ ]:
# Kinldy be sure that the evaluation tool trec_eval is installed. Instructions 
# can be found at: https://github.com/usnistgov/trec_eval
# Provide the  absolute path of the tool here:
TREC_EVAL_PATH = "/trec_eval"


In [ ]:
BATCH_SIZE = 64
EPOCHS = 5
MODEL_DIR = ''
LOG_DIR = ''
LAGRANGE_MULTIPLIER = 0.02
INPUT_DATA_PATH = ''
Supervised_files_PATH = 'Supervised_ClickNRR_files/'
Embeddings_Index_PATH = 'Embeddings_Index_files/'
MODEL_NAME = 'crm_clicks_logs'
BANDIT_FEEDBACK_UPDATE = 10000 #number of bandit-feedback for model update

In [ ]:
# Load data
train_data = pd.read_csv(os.path.join(INPUT_DATA_PATH, 'logs/click_logs.csv'))
dev_data = pd.read_csv(os.path.join(Supervised_files_PATH, 'Supervised_Dev_ClickNRR_File.csv'))
test_data = pd.read_csv(os.path.join(Supervised_files_PATH, 'Supervised_Test_ClickNRR_File.csv'))

qids_train = train_data['qid']
loss_train = train_data['loss']
action_train = train_data['action']
addn_feat_train = np.array(train_data['addn_feat'].apply(lambda x: [float(elem) for elem in x.split(', ')]).tolist())
probs_train = train_data['control_policy_prob']

qids_dev = dev_data['qids']
addn_feat_dev = np.array(dev_data['addn_feat'].apply(lambda x: [float(elem) for elem in x.split(', ')]).tolist())

qids_test = test_data['qids']
addn_feat_test = np.array(test_data['addn_feat'].apply(lambda x: [float(elem) for elem in x.split(', ')]).tolist())

# Load embeddings matrix
embeddings = np.load(os.path.join(INPUT_DATA_PATH, 'embedding.npy'))

# Load embeddings indices for queries and product titles

# Each row in q_train corresponds to the query of the sample in train_data. Both are aligned.
# q_train contain numpy arrays of query word indices used for lookup in embedding matrix. 
q_train = np.load(os.path.join(Embeddings_Index_PATH, 'click_logs_queries.npy'))

# Each row in a_train corresponds to the title of product in train_data. Both are aligned.
# a_train contain numpy arrays of word indices of the product title, these indices are used for lookup in embedding matrix. 
a_train = np.load(os.path.join(Embeddings_Index_PATH, 'click_logs_products.npy'))

q_dev = np.load(os.path.join(Embeddings_Index_PATH, 'queries_dev.npy'))
a_dev = np.load(os.path.join(Embeddings_Index_PATH, 'products_dev.npy'))

q_test = np.load(os.path.join(Embeddings_Index_PATH, 'queries_test.npy'))
a_test = np.load(os.path.join(Embeddings_Index_PATH, 'products_test.npy'))


In [ ]:
# Choose variant of relevance label here:

# binary-ceiled relevance label
y_dev = np.ceil(dev_data['click_NRR']).astype(int)
y_test = np.ceil(test_data['click_NRR']).astype(int)

# binary-roundedoff relevance label

# y_dev = np.round(dev_data['click_NRR']).astype(int)
# y_test = np.round(test_data['click_NRR']).astype(int)

# graded-ceiled relevance label

# max_relevance_grade = 4
# y_dev = np.ceil(dev_data['click_NRR']*max_relevance_grade).astype(int)
# y_test = np.ceil(test_data['click_NRR']*max_relevance_grade).astype(int)

In [ ]:
# Fixing random seed for ease of reproducibility of results
np.random.seed(2) 
random.seed(2) 
set_random_seed(2)

addit_feat_len = addn_feat_train.shape[1]
embed_dim = embeddings.shape
max_ques_len = q_train.shape[1]
max_ans_len = a_train.shape[1]

with tf.Graph().as_default():
    with tf.Session() as sess:
        set_session(sess)

        # Create instance of model
        cnn_model_instance = crm_model(max_ques_len, max_ans_len,  embeddings, addit_feat_len=addit_feat_len)

        # Compute weights for the model
        weights_train = (loss_train - LAGRANGE_MULTIPLIER)/probs_train 

        y_pred_dev = cnn_model_instance.predict([q_dev, a_dev, addn_feat_dev, np.ones(shape = len(q_dev))])
        map_score_overall, mrr, p_5, p_10, ndcg_5, ndcg_10 = get_trec_eval_metrics(qids_dev, y_pred_dev, y_dev, TREC_EVAL_PATH)
        print('Initial results on {} set are: MAP: {}, MRR:{}, P@5: {}, P@10: {}, NDCG@5: {}, NDCG@10: {}'
              .format('DEV', map_score_overall, mrr, p_5, p_10, ndcg_5, ndcg_10))
        y_pred_test = cnn_model_instance.predict([q_test, a_test, addn_feat_test, np.ones(shape = len(q_test))])
        map_score, mrr, p_5, p_10, ndcg_5, ndcg_10 = get_trec_eval_metrics(qids_test, y_pred_test, y_test, TREC_EVAL_PATH)
        print('Initial results on {} set are: MAP: {}, MRR:{}, P@5: {}, P@10: {}, NDCG@5: {}, NDCG@10: {}'
              .format('TEST', map_score, mrr, p_5, p_10, ndcg_5, ndcg_10))

        best_model_weights = cnn_model_instance.get_weights()
        ndcg_10_overall = 0
        
        for epoch in range(EPOCHS):
            print('Epoch: ', epoch)

            BANDIT_FEEDBACK = 0
            for b_q_train, b_a_train, b_addn_feat_train, b_weights, b_action, b_qid in zip(
                batch_gen(q_train, BATCH_SIZE), batch_gen(a_train, BATCH_SIZE), 
                batch_gen(addn_feat_train, BATCH_SIZE), batch_gen(weights_train, BATCH_SIZE),
                batch_gen(action_train, BATCH_SIZE), batch_gen(qids_train, BATCH_SIZE)):

                cnn_model_instance.train_on_batch([b_q_train, b_a_train, b_addn_feat_train, b_weights], b_action)
                
                BANDIT_FEEDBACK += 1
                if BANDIT_FEEDBACK%BANDIT_FEEDBACK_UPDATE == 0:

                    print('{} BANDIT FEEDBACKS were already processed'.format(BANDIT_FEEDBACK*BATCH_SIZE))   

                    y_pred_dev = cnn_model_instance.predict([q_dev, a_dev, addn_feat_dev, np.ones(shape = len(q_dev))])
                    map_score, mrr, p_5, p_10, ndcg_5, ndcg_10 = get_trec_eval_metrics(qids_dev, y_pred_dev, y_dev, TREC_EVAL_PATH)
                    if ndcg_10 > ndcg_10_overall:
                        print ('Model Weights Updated')
                        print ('Results on {} set are: MAP: {}, MRR:{}, NDCG@5: {}, NDCG@10: {}'.format('DEV', map_score, mrr, ndcg_5, ndcg_10))
                        ndcg_10_overall = ndcg_10
                        best_model_weights = cnn_model_instance.get_weights()

        cnn_model_instance.set_weights(best_model_weights)
        cnn_model_instance.save(os.path.join(MODEL_DIR, MODEL_NAME+'.h5'))

        y_pred_dev = cnn_model_instance.predict([q_dev, a_dev, addn_feat_dev, np.ones(shape = len(q_dev))])
        map_score, mrr, p_5, p_10, ndcg_5, ndcg_10 = get_trec_eval_metrics(qids_dev, y_pred_dev, y_dev, TREC_EVAL_PATH)
        print('BEST results on {} set are: MAP: {}, MRR:{}, P@5: {}, P@10: {}, NDCG@5: {}, NDCG@10: {}'
              .format('DEV', map_score, mrr, p_5, p_10, ndcg_5, ndcg_10))
        y_pred_test = cnn_model_instance.predict([q_test, a_test, addn_feat_test, np.ones(shape = len(q_test))])
        map_score, mrr, p_5, p_10, ndcg_5, ndcg_10 = get_trec_eval_metrics(qids_test, y_pred_test, y_test, TREC_EVAL_PATH)
        print('BEST results on {} set are: MAP: {}, MRR:{}, P@5: {}, P@10: {}, NDCG@5: {}, NDCG@10: {}'
              .format('TEST', map_score, mrr, p_5, p_10, ndcg_5, ndcg_10))